In [1]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Import classes from Class.py
from Data_handling import *
from Class import *
from Class_no_zone import *

In [2]:
## DATA INDEX

#Data_df

In [ ]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    

epsilon= 0.1
alpha = 0.95
beta = 0
# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_OpCost)   # Number of existing generators
N_gen_N = len(Gen_N_OpCost)   # Number of new generators
N_zone = len(Trans_Z_Connected_To_Z)     # Number of zones
N_line = len(Trans_Line_From_Z)   # Number of transmission lines


# Hyperparameters
B = 1000000000   # Budget for the investment problem
R = 73 # Conversion rate


In [4]:
## CREATE THE PARAMETERS AND DATA OBJECTS
ParametersObj = Parameters(H, D, Y, N, N_dem, N_gen_E, N_gen_N, N_zone, N_line, B, R, N_S, N_S_test, max_deviation,epsilon, alpha, beta)
DataObj = InputData(Dem, Uti, Load_Z, Gen_E_OpCost, Gen_N_OpCost, Gen_N_MaxInvCap, Gen_E_Cap, Gen_N_InvCost, Gen_E_Tech, Gen_N_Tech, Gen_E_Z, Gen_N_Z, Gen_E_OpCap, Gen_N_OpCap, Trans_React, Trans_Cap, Trans_Line_From_Z, Trans_Line_To_Z, Trans_Z_Connected_To_Z,Gen_N_Data_scenarios,Gen_N_OpCost_scenarios)

# Model 1: Sequential optimization of Dispatch problem and Investment problem

### 1) Market Clearing

In [5]:
# Run the Market Clearing Problem
MarketClearing1 = MarketClearingModel1(ParametersObj, DataObj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-24


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 140400 rows, 126000 columns and 270000 nonzeros
Model fingerprint: 0x4c1471a4
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [1e+03, 1e+03]
  RHS range        [4e-01, 1e+03]
Presolve removed 134239 rows and 109069 columns
Presolve time: 1.18s
Presolved: 6161 rows, 16931 columns, 19670 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.739e+03
 Factor NZ  : 8.900e+03 (roughly 10 MB of memory)
 Factor Ops : 1.438e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0 

In [6]:
#MarketClearing1.res.df.head(10)

### 2) Investment Problem

In [7]:
# Run the investmentmodel
InvestmentPB1 = InvestmentModel1(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57857 rows, 57616 columns and 111932 nonzeros
Model fingerprint: 0xbaa5e9fc
Coefficient statistics:
  Matrix range     [4e-04, 1e+06]
  Objective range  [1e+03, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+09]
Presolve removed 57856 rows and 57610 columns
Presolve time: 0.10s
Presolved: 1 rows, 6 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2734490e+10   3.072000e+05   0.000000e+00      0s
       1    2.1889150e+10   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.15 seconds (0.06 work units)
Optimal objective  2.188915000e+10
Objective value:  21889150000.0


In [8]:
InvestmentPB1.res.df

,Technology,Invested capacity (MW)
0,Coal,0.0
1,Coal,0.0
2,Coal,0.0
3,Gas,0.0
4,Coal,0.0
5,Wind,400.0
6,Wind,200.0
7,PV,0.0
8,Gas,0.0
9,Coal,0.0


### 3) Stochastic robust investment problem

In [9]:
#Run the stochastic investment model (without scenario dependent variables)
Investment_Robust = InvestmentModel_Robust(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57857 rows, 57616 columns and 111932 nonzeros
Model fingerprint: 0xb0375178
Coefficient statistics:
  Matrix range     [4e-04, 1e+06]
  Objective range  [1e+03, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+09]
Presolve removed 57856 rows and 57600 columns
Presolve time: 0.14s
Presolved: 1 rows, 16 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5903199e+10   3.207565e+05   0.000000e+00      0s
       1    2.8896986e+10   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.20 seconds (0.05 work units)
Optimal objective  2.889698625e+10
Objective value:  28896986252.011463


In [10]:
Investment_Robust.res.df

,Technology,Invested capacity (MW)
0,Coal,0.000000
1,Coal,0.000000
2,Coal,0.000000
3,Gas,0.000000
4,Coal,0.000000
5,Wind,400.000000
6,Wind,400.000000
7,PV,386.482998
8,Gas,0.000000
9,Coal,0.000000


In [11]:
#Run the stochastic investment model (WITH scenario dependent variables)
Investment_Stochastic = InvestmentModel_Stochastic(ParametersObj, DataObj, MarketClearing1.res.DA_price, )

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57857 rows, 57616 columns and 111932 nonzeros
Model fingerprint: 0x4a2a551c
Coefficient statistics:
  Matrix range     [4e-04, 1e+06]
  Objective range  [2e+05, 2e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+09]
Presolve removed 57856 rows and 57600 columns
Presolve time: 0.16s
Presolved: 1 rows, 16 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1156427e+14   2.225757e+05   0.000000e+00      0s
       1    1.4654348e+14   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.24 seconds (0.06 work units)
Optimal objective  1.465434753e+14
Objective value:  146543475336167.0


In [13]:
Investment_Stochastic.res.df

,Technology,Invested capacity (MW)
0,Coal,400.000000
1,Coal,0.000000
2,Coal,198.380094
3,Gas,400.000000
4,Coal,0.000000
5,Wind,0.000000
6,Wind,400.000000
7,PV,0.000000
8,Gas,0.000000
9,Coal,0.000000


In [14]:
ParametersObj.N_S

1

In [15]:
DataObj.Gen_N_Data_scenarios.shape

(16, 1)

### 3) Chance Constraint

In [16]:
Investment_Stochastic_ChanceConstraint = InvestmentModel_Stochastic_ChanceConstraint(ParametersObj, DataObj, MarketClearing1.res.DA_price)

KeyboardInterrupt: 

In [ ]:
Investment_Stochastic_ChanceConstraint.res.df

,Technology,Invested capacity (MW)
0,Coal,0.000000
1,Coal,45.161250
2,Coal,0.000000
3,Gas,0.000000
4,Coal,0.000000
5,Wind,142.914022
6,Wind,400.000000
7,PV,0.000000
8,Gas,0.000000
9,Coal,0.000000


### 4) CVaR Model

In [ ]:
Investment_CVaR = InvestmentModel_CVaR(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 57862 rows, 57620 columns and 284818 nonzeros
Model fingerprint: 0x2754130b
Coefficient statistics:
  Matrix range     [4e-04, 1e+06]
  Objective range  [4e+02, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+09]
Presolve removed 57859 rows and 57604 columns
Presolve time: 0.08s
Presolved: 3 rows, 16 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3871188e+10   6.849189e+05   0.000000e+00      0s
       3    1.9760421e+10   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.14 seconds (0.10 work units)
Optimal objective  1.976042081e+10
Objective value:  19760420807.63359


In [ ]:
Investment_CVaR.res.df

,Technology,Invested capacity (MW)
0,Coal,0.000000
1,Coal,45.161250
2,Coal,0.000000
3,Gas,0.000000
4,Coal,0.000000
5,Wind,142.914022
6,Wind,400.000000
7,PV,0.000000
8,Gas,0.000000
9,Coal,0.000000


In [ ]:
# Export the results into the excel file called Investment Results and Market Clearing Results which are located in the folder Results
#InvestmentPB1.res.df.to_excel("../Results/InvestmentResults.xlsx")
#MarketClearing1.res.df.to_excel("../Results/MarketClearingResults.xlsx")

# Model 2: Integrated Bi-level optimization of dispatch problem and investment problem using KKTs

In [ ]:
# ## CREATE THE PARAMETERS AND DATA OBJECTS
# ParametersObj_SHORT = Parameters(H, D, Y, 10, N_dem, N_gen_E, N_gen_N, N_zone, N_line, B, R)
# DataObj_SHORT = InputData(Dem[0:10,:], Uti, Load_Z, Gen_E_OpCost, Gen_N_OpCost, Gen_N_MaxInvCap, Gen_E_Cap, Gen_N_InvCost, Gen_E_Tech, Gen_N_Tech, Gen_E_Z, Gen_N_Z, Gen_E_OpCap[0:10,:], Gen_N_OpCap[0:10,:], Trans_React, Trans_Cap, Trans_Line_From_Z, Trans_Line_To_Z, Trans_Z_Connected_To_Z)

In [ ]:
# class Model_2_no_zone():
#     def __init__(self, ParametersObj, DataObj, Model_results = 1, Guroby_results = 1):
#         self.P = ParametersObj # Parameters
#         self.D = DataObj # Data
#         self.Model_results = Model_results
#         self.Guroby_results = Guroby_results
#         self.var = Expando()  # Variables
#         self.con = Expando()  # Constraints
#         self.res = Expando()  # Results
#         self._build_model() 


#     def _build_variables(self):
#         self.var.P_N = self.m.addMVar((self.P.N_gen_N, 1), lb=0) # Invested capacity in every new generator
#         self.var.d = self.m.addMVar((self.P.N, self.P.N_dem), lb=0)  # demand per hour for every load
#         self.var.p_E = self.m.addMVar((self.P.N, self.P.N_gen_E), lb=0)  # power output per hour for every existing generator
#         self.var.p_N = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0) # Power output per hour for every new generator
        
#         # Dual variables
#         self.var.DA_Price = self.m.addMVar((self.P.N, 1), lb=-GRB.INFINITY)  # Day ahead price per hour
#         self.var.mu_E_up = self.m.addMVar((self.P.N, self.P.N_gen_E), lb=0)  # Dual 1
#         self.var.mu_E_down = self.m.addMVar((self.P.N, self.P.N_gen_E), lb=0)  # Dual 1
#         self.var.mu_N_up = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0)  # Dual 2
#         self.var.mu_N_down = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0)  # Dual 2
#         self.var.nu_up = self.m.addMVar((self.P.N, self.P.N_dem), lb=0)  # Dual 3
#         self.var.nu_down = self.m.addMVar((self.P.N, self.P.N_dem), lb=0)  # Dual 3
        
        

#     def _build_constraints(self):
#         # Capacity investment constraint
#         self.con.cap_inv = self.m.addConstr(self.var.P_N <= self.D.Gen_N_MaxInvCap, name='Maximum capacity investment')

#         # Budget constraint
#         self.con.budget = self.m.addConstr(self.var.P_N.T @ self.D.Gen_N_InvCost <= self.P.B, name='Budget constraint')

#         ## PRIMAL CONSTRAINTS
#         # Max production constraint existing
#         self.con.max_p_E = self.m.addConstr(self.var.p_E <= self.D.Gen_E_OpCap * (self.P.Sum_over_hours @ self.D.Gen_E_Cap.T), name='Maximum production of existing generators')

#         # Max production constraint new, in a different shape because we can't transpose varaibles
#         self.con.max_p_N = self.m.addConstrs(self.var.p_N[h, g] <= (self.D.Gen_N_OpCap[h, g] *  self.var.P_N[g].T) for h in range(self.P.N) for g in range(self.P.N_gen_N))

#         # Max demand constraint
#         self.con.max_dem = self.m.addConstr(self.var.d <= self.D.Dem, name='Maximum demand')        

#         # Balance constraint
#         prod_E = self.var.p_E @ self.P.Sum_over_gen_E  
#         prod_N = self.var.p_N @ self.P.Sum_over_gen_N  
#         dem = self.var.d @ self.P.Sum_over_dem
#         self.con.balance = self.m.addConstr(dem == prod_E + prod_N, name='Power balance') 

#         ## FIRST ORDER CONDITIONS

#         self.con.L_p_EC = self.m.addConstr(self.P.Sum_over_hours @ self.D.Gen_E_OpCost.T - self.var.DA_Price @ self.P.Sum_over_gen_E.T - self.var.mu_E_up + self.var.mu_E_down == 0, name='L_p_EC')
#         self.con.L_p_NC = self.m.addConstr(self.P.Sum_over_hours @ self.D.Gen_N_OpCost.T - self.var.DA_Price @ self.P.Sum_over_gen_N.T - self.var.mu_E_up + self.var.mu_E_down == 0, name='L_p_NC')
#         self.con.L_d = self.m.addConstr(- self.P.Sum_over_hours @ self.D.Uti.T + self.var.DA_Price @ self.P.Sum_over_dem.T - self.var.nu_up + self.var.nu_down == 0, name='L_d')
        

#         ## COMPLMEENTARY CONDITIONS

#         # Define the Big M matrixes

#         Big_M_p = np.full((self.P.N, self.P.N_gen_E), 1e3)  # Big M for the production constraints
#         Big_M_d = np.full((self.P.N, self.P.N_dem), 1e3)  # Big M for the demand constraints

#         # Define the binary variables

#         b_E_down = self.m.addVars(self.P.N, self.P.N_gen_E, vtype=gp.GRB.BINARY, name="b_E_down")
#         b_E_up = self.m.addVars(self.P.N, self.P.N_gen_E, vtype=gp.GRB.BINARY, name="b_E_up")

#         b_N_down = self.m.addVars(self.P.N, self.P.N_gen_N, vtype=gp.GRB.BINARY, name="b_N_down")
#         b_N_up = self.m.addVars(self.P.N, self.P.N_gen_N, vtype=gp.GRB.BINARY, name="b_N_up")

#         b_d_down = self.m.addVars(self.P.N, self.P.N_dem, vtype=gp.GRB.BINARY, name="b_d_down")
#         b_d_up = self.m.addVars(self.P.N, self.P.N_dem, vtype=gp.GRB.BINARY, name="b_d_up")

#         # Define the complementary constraints
#         # Existing generators
#         self.con.compl_E_down_mu = self.m.addConstrs((self.var.mu_E_down[h, g] <= Big_M_p[h, g] * b_E_down[h, g] for h in range(self.P.N) for g in range(self.P.N_gen_E)), name='compl_E_down_mu')
#         self.con.compl_E_down_p = self.m.addConstrs((self.var.p_E[h, g] <= Big_M_p[h, g] * (1 - b_E_down[h, g]) for h in range(self.P.N) for g in range(self.P.N_gen_E)), name='compl_E_down_p')
#         self.con.compl_E_up_mu = self.m.addConstrs((self.var.mu_E_up[h, g] <= Big_M_p[h, g] * b_E_up[h, g] for h in range(self.P.N) for g in range(self.P.N_gen_E)), name='compl_E_up_mu')
#         self.con.compl_E_up_p = self.m.addConstrs(((self.D.Gen_E_OpCap[h, g] - self.var.p_E[h, g]) <= Big_M_p[h, g] * (1 - b_E_up[h, g]) for h in range(self.P.N) for g in range(self.P.N_gen_E)), name='compl_E_up_p')

#         # New generators
#         self.con.compl_N_down_mu = self.m.addConstrs((self.var.mu_N_down[h, g] <= Big_M_p[h, g] * b_N_down[h, g] for h in range(self.P.N) for g in range(self.P.N_gen_N)), name='compl_N_down_mu')
#         self.con.compl_N_down_p = self.m.addConstrs((self.var.p_N[h, g] <= Big_M_p[h, g] * (1 - b_N_down[h, g]) for h in range(self.P.N) for g in range(self.P.N_gen_N)), name='compl_N_down_p')
#         self.con.compl_N_up_mu = self.m.addConstrs((self.var.mu_N_up[h, g] <= Big_M_p[h, g] * b_N_up[h, g] for h in range(self.P.N) for g in range(self.P.N_gen_N)), name='compl_N_up_mu')
#         self.con.compl_N_up_p = self.m.addConstrs(((self.D.Gen_N_OpCap[h, g] - self.var.p_N[h, g]) <= Big_M_p[h, g] * (1 - b_N_up[h, g]) for h in range(self.P.N) for g in range(self.P.N_gen_N)), name='compl_N_up_p')

#         # Demand
#         self.con.compl_d_down_mu = self.m.addConstrs((self.var.nu_down[h, d] <= Big_M_d[h, d] * b_d_down[h, d] for h in range(self.P.N) for d in range(self.P.N_dem)), name='compl_d_down_mu')
#         self.con.compl_d_down_p = self.m.addConstrs((self.var.d[h, d] <= Big_M_d[h, d] * (1 - b_d_down[h, d]) for h in range(self.P.N) for d in range(self.P.N_dem)), name='compl_d_down_p')
#         self.con.compl_d_up_mu = self.m.addConstrs((self.var.nu_up[h, d] <= Big_M_d[h, d] * b_d_up[h, d] for h in range(self.P.N) for d in range(self.P.N_dem)), name='compl_d_up_mu')
#         self.con.compl_d_up_p = self.m.addConstrs(((self.D.Dem[h, d] - self.var.d[h, d]) <= Big_M_d[h, d] * (1 - b_d_up[h, d]) for h in range(self.P.N) for d in range(self.P.N_dem)), name='compl_d_up_p')




#     def _build_objective(self):

#         # 1. Costs related to existing generators (G_EC)
#         cost_EC_p = (self.var.p_E @ self.D.Gen_E_OpCost).sum()
#         cost_EC_mu = ((self.D.Gen_E_OpCap * (self.P.Sum_over_hours @ self.D.Gen_E_Cap.T)) * self.var.mu_E_up).sum()

#         # 2. Benefits and costs related to demand (D)
#         benefit_d = (self.var.d @ self.D.Uti).sum()
#         cost_nu_d = (self.D.Dem * self.var.nu_up).sum()

#         # 3. Costs related to new generators (G_NC)
#         cost_NC_p = (self.var.p_N @ self.D.Gen_N_OpCost).sum()
#         cost_NC_P = (self.var.P_N.T @ self.D.Gen_N_InvCost).sum()

#         # Full objective function
#         self.m.setObjective(
#             - cost_EC_p - cost_EC_mu  # Existing generators
#             + benefit_d - cost_nu_d   # Demand
#             - cost_NC_p - cost_NC_P,  # New generators
#             gp.GRB.MAXIMIZE
#         )

        


#     def _display_guropby_results(self):
#         self.m.setParam('OutputFlag', self.Guroby_results)
#         #self.m.setParam('Method', 2)  # Use barrier method
#         #self.m.setParam('Crossover', 0)  # Skip crossover for speed
#         #self.m.setParam('Heuristics', 0.2)  # Enable aggressive heuristics
#         #self.m.setParam('MIPFocus', 1)  # Focus on finding feasible solutions
    

#     def _build_model(self):
#         self.m = gp.Model('Model 2')
#         self._build_variables()  
#         self._build_constraints()
#         self._build_objective()
#         self._display_guropby_results()
#         self.m.optimize()
#         if self.Model_results == 1:
#             self._extract_results()

#     def _extract_results(self):
#         # Display the objective value
#         print('Objective value: ', self.m.objVal)
        
#         # Display the generators the model invested in, in a dataframe
#         self.res.P_N = self.var.P_N.X
#         self.res.P_N = self.res.P_N.reshape((self.P.N_gen_N,1))
#         self.res.df = pd.DataFrame(self.D.Gen_N_Tech, columns = ['Technology'])
#         self.res.df['Invested capacity (MW)'] = self.res.P_N
            

        

In [ ]:
# Model2NZ = Model_2_no_zone(ParametersObj_SHORT, DataObj_SHORT)